In [1]:
from pathlib import Path
import requests

BUNKACHOU_URL = 'https://www.bunka.go.jp/seisaku/kokugo_nihongo/kokugo_shisaku/kikigengo/archivejigyo/pdf/kikigengo_chiba{id:02}.pdf'

TEXT_PDF_DIR = Path('../../../../語言/東亞語/阿伊努語/文化庁アーカイブ')

def download():
    if not TEXT_PDF_DIR.exists():
        TEXT_PDF_DIR.mkdir()
    
    for i in range(1, 3 + 1):
        url = BUNKACHOU_URL.format(id=i)
        print(url)
        r = requests.get(url)
        target = TEXT_PDF_DIR / f'chiba{i:02}.pdf'
        if target.exists():
            print(f'{target} already exists')
            continue
        with open(target, 'wb') as f:
            f.write(r.content)
    
download()

https://www.bunka.go.jp/seisaku/kokugo_nihongo/kokugo_shisaku/kikigengo/archivejigyo/pdf/kikigengo_chiba01.pdf
..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba01.pdf already exists
https://www.bunka.go.jp/seisaku/kokugo_nihongo/kokugo_shisaku/kikigengo/archivejigyo/pdf/kikigengo_chiba02.pdf
..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba02.pdf already exists
https://www.bunka.go.jp/seisaku/kokugo_nihongo/kokugo_shisaku/kikigengo/archivejigyo/pdf/kikigengo_chiba03.pdf
..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba03.pdf already exists


In [2]:
for f in TEXT_PDF_DIR.glob("*.pdf"):
    print(f)

..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba01.pdf
..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba02.pdf
..\..\..\..\語言\東亞語\阿伊努語\文化庁アーカイブ\chiba03.pdf


In [3]:
from pypdf import PdfReader, PageObject
import json
import regex as re
from tqdm.notebook import tqdm

TEXT_TXT_DIR = Path("./data/chiba/")

TOC_PAGE_RANGE = [(1, 3), (1, 2), (1, 2)]

result = []

Range = tuple[int, int]


def read_page_content(page: PageObject, y_range: Range) -> str:
    parts = []

    y_min, y_max = y_range

    def visitor_body(text, cm, tm, font_dict, font_size):
        y = tm[5]
        if y > y_min and y < y_max:
            parts.append(text)

    page.extract_text(visitor_text=visitor_body)

    return "".join(parts)


def read_pages(pages: list[PageObject], page_range: Range, y_range: Range) -> str:
    pages_content = []
    for page in pages[page_range[0] - 1 : page_range[1]]:
        pages_content.append(read_page_content(page, y_range))
    return "\n".join(pages_content)


last_number_of_pages = 0
main_bar = tqdm(list(enumerate(TEXT_PDF_DIR.glob("*.pdf"))), desc="Reading PDFs")
for i, pdf_file in main_bar:
    reader = PdfReader(pdf_file)
    number_of_pages = len(reader.pages)
    main_bar.set_description(f"Reading PDF {pdf_file.name} ({number_of_pages} pages)")

    toc_page = read_pages(reader.pages, TOC_PAGE_RANGE[i], (50, 720))

    texts = []
    TOC_PATTERN = re.compile(r"^(\d+-\d+)\s(.+?)[\s·]+(\d+) $", flags=re.MULTILINE)
    content_toc_lines = [
        m
        for line in toc_page.splitlines()
        if "「" in line and (m := re.match(TOC_PATTERN, line))
    ]

    for j, line in enumerate(content_toc_lines):
        id, title, page = line.groups()

        page = int(page) - last_number_of_pages
        next_page = (
            int(content_toc_lines[j + 1].groups()[2]) - last_number_of_pages
            if j + 1 < len(content_toc_lines)
            else number_of_pages + last_number_of_pages - 2 * (i + 1)
        )

        page_range = (page, next_page)

        title = re.sub(r"\s+", " ", title)

        texts.append((id, title, page_range))

    last_number_of_pages += number_of_pages
    bar = tqdm(
        texts,
        desc=f"Reading text in {pdf_file.name}",
    )
    for id, title, (start, end) in bar:
        bar.set_description(f"Reading text in {pdf_file.name} ({id}, pp.{start}-{end})")

        if any(title.endswith(e) for e in ["解説", "説明", "紹介"]):
            continue

        page = read_pages(reader.pages, (start, end), (50, 790)).strip()
        page = "\n".join(map(lambda x: x.strip(), page.splitlines()))

        page = re.sub(r"[（\()][?？][）\)]", "", page)  # (?) （？）

        # Remove line with only parenthesis
        page = re.sub(r"^[（\(][^）\)]+[）\)]$", "", page, flags=re.MULTILINE)  # （萱野：うんうん）
        # Remove line with only numbers (wrongly parsed nombre)
        page = re.sub(r"^\d+$", "", page, flags=re.MULTILINE)  # 1

        #     if id == '1-4':
        #         print(page)
        #         break
        # break

        # print('#' * 80)
        # print(page)
        # print('#' * 80)
        # break

        try:
            full_title, authorship, content = map(
                lambda x: x.strip(),
                re.split(r"\n{2,}", page, maxsplit=2),
            )
        except ValueError:
            print("=== ERROR ===")
            print(pdf_file.name)
            print(id, title)
            print(page)
            print(start, end)
            print("=============")
            break

        # " 」" -> "」"
        full_title = re.sub(r"\s+」", "」", full_title)

        title_translation = full_title.splitlines()[-1]

        # print(title_translation)

        # print('...')
        authors = {}
        for line in authorship.splitlines():
            if not "：" in line:
                continue
            try:
                t, n = line.rsplit("：", 1)
            except ValueError:
                print("=== ERROR ===")
                print(pdf_file.name)
                print(id, title)
                print()
                print(line)
                print("=============")
                break

            if t == "語り手":
                t = "語り"
            authors[t] = n.replace(" ", "")

        # Remove anything after next chapter: "1968  20-8"
        content = re.split(r"\d+\s+\d+-\d+", content, maxsplit=1)[0]
        content = content.strip()

        try:
            text, footnote = (part.strip() for part in content.split("【注】"))
        except ValueError:
            text, footnote = content, None

        # for i, line in enumerate(content.splitlines()):
        #     print(f'{i:02d}', line)
        text = re.sub(r"\n{3,}", "\n\n", text)

        # Remove numbers at the beginning of lines
        text = re.sub(r"^\d+\s+", "", text, flags=re.MULTILINE)

        # Remove footnote inline reference (e.g. [1])
        text = re.sub(r"\[\d+\]", "", text)

        # Remove space before underscore (h _ine -> h_ine)
        text = re.sub(r"(\w)\s+_", r"\1_", text)

        # Fix separated = (e.g. "a =eci=epunkine" -> "a=eci=epunkine")
        if " =" in text:
            text = text.replace(" =", "=")
        if "= " in text:
            text = text.replace("= ", "=")

        # Fix separated "y" (e.g.
        #   "C y akne" -> "yakne", (C y V -> C yV)
        #   "su y hetopo" -> "suy hetopo" (V y C -> Vy C)
        #   TODO: "kamu y utarpa" -> "kamuy utarpa", "siknu y akun" -> "siknuy akun"
        # )
        CONSONANT = r"[^aeiou]"
        VOWEL = r"[aeiou]"
        text = re.sub(rf"({CONSONANT})\s+y\s+({VOWEL})", r"\1 y\2", text)
        text = re.sub(rf"({VOWEL})\s+y\s+({CONSONANT})", r"\1y \2", text)
        # C s V -> C sV
        text = re.sub(rf"({CONSONANT})\s+s\s+({VOWEL})", r"\1s\2", text)

        # Fix special cases:
        if id == "15-15":
            text = text.replace("ヤイェイヌ  クス ネ アクス\n", "")
            text = text.replace("ヤイイェイヌ クス ネ アクス", "ヤイェイヌ クス ネ アクス")

        if id == "13-2":
            text = text.replace("アイウタ ……\nayuta..\n.", "")

        if id == "3-6":
            text = text.replace("二次的なサケヘ", "")

        title = title.strip()

        if "ウラユシウンクㇽ" in title:
            title = "ウエペケㇾ「ウラユシウンクㇽ」"
            title_translation = "ウラユシの人"

        if "ラ ンコ カッケマッ " in title:
            title = "ウエペケㇾ「ランコ カッケマッ」"

        if title == "ウエペケㇾ「スルクマ ッ チクペニカムイ イカオピューキ」":
            title = "ウエペケㇾ「スルクマッ チクペニカムイ イカオピューキ」"

        if title == "ルパイェユカㇻ「ウェンク ㇽ フチ イレス」":
            title = "ルパイェユカㇻ「ウェンクㇽ フチ イレス」"

        # Remove immediate space after "「" or before "」"
        title = re.sub(r"\s*「\s*", "「", title)
        title = re.sub(r"\s*」\s*", "」", title)

        title = re.sub(r"\s*（\s*", "（", title)
        title = re.sub(r"\s*）\s*", "）", title)

        # if "フㇺ パㇰパㇰ" in title:
        #     title = "カムイユカㇻ「フㇺ パㇰパㇰ」"

        result.append(
            {
                "id": id,
                "title": title,
                "title_translation": title_translation,
                "authors": authors,
                "content": text,
                "footnote": footnote,
                "book_index": i + 1,
            }
        )

        # metadata = re.match(r"^(.+)\n\n(.*)\n\n", '\n'.joi`n(metadata_lines), flags=re.DOTALL)
        # print(metadata)

        # print('-' * 80)

        # tit

        # speaker = content_lines[4].split('：')[1]
        # print(title_translation, speaker)
        # content_lines = content.splitlines()[5:]

        # content_lines = [line.strip() for line in content_lines if line.strip() != '']
        # for line in content_lines:
        #     print(line)
        # break


with open("corpus/chiba/chiba.json", "w") as f:
    json.dump(result, f, ensure_ascii=False)

Reading PDFs:   0%|          | 0/3 [00:00<?, ?it/s]

Reading text in chiba01.pdf:   0%|          | 0/58 [00:00<?, ?it/s]

Reading text in chiba02.pdf:   0%|          | 0/45 [00:00<?, ?it/s]

Reading text in chiba03.pdf:   0%|          | 0/42 [00:00<?, ?it/s]

In [5]:
pdf_file = list(TEXT_PDF_DIR.glob("*.pdf"))[0]
reader = PdfReader(pdf_file)
number_of_pages = len(reader.pages)
print(f"{pdf_file.name} ({number_of_pages} pages)")
print('=' * 80)

reader = PdfReader(pdf_file)
reader.pages[33].extract_text()

print(read_page_content(reader.pages[34 - 1], (50, 790)))
print('=' * 80)

chiba01.pdf (828 pages)

 ピㇼカ  カトゥ  ピㇼカ  メノコ  ネ ルウェ  ネ ナ、ポンマッ  ネ エトゥン  ヤン。  
pirka katu pirka menoko ne ruwe ne na,  ponmat ne  etun yan.   
容姿の良い 美しい女性ですよ。妾として 貰ってください。  
 
ヤㇰネ  ポ コㇿ ヤㇰネ  イコイヨマㇷ゚  ポカ アン クㇱ ネ ナ」 
yakne po kor y akne ikoyomap poka  =an kus ne na. ” 
それで子どもが出来 たならば （その子を）かわいがることだけでも出来るのですから。 」 
 
ㇱコㇿ カネ ハウェアン  ルウェ  ネ コㇿカ  アエランポキウェン   
sekor kane hawean ruwe  ne korka a=erampokiwen  
そのように言います。しかしそれでは かわいそうだ  
 
アイルㇱカレ  カ エランポキウェン  クㇱ アコソモタㇱヌ  ワ  
a=iruskare ka  erampokiwen kus  nena  a=kosomotasnu wa  
彼女に腹立たしい思いをさせたりするの もかわいそうだと思い 知らぬふりをして  
 
アナナ  ㇷ゚ コント  イペ カ ソモ キ ノ チㇱ コㇿ イェ ヒ クス  
an=an a p konto ipe ka  somo ki no cis kor ye hi kusu  
いたのですが、しまいには食事もとらずに 泣きながら （そのことを） 言うので  
 
オラウン  アエトゥン  アクス  ポン ユピヒ  ポロ ユピヒ  オナハ  ウヌフ  アン 
oraun a=etun akusu pon yupihi  poro yupihi onaha unuhu an  
とうとう妾を 貰いました。 下の兄、上の兄、父親、母親と その人にはいたのですが、  
 
ペ ネ ㇷ゚ ラムオㇱマ  パ ヒネ マッ ネ アエトゥン  ヒネ ポン マッ ネ 
pe ne p ramuosma pa hine mat ne a=etun h ine pon mat ne  
皆同意してくれたので 妻として 、妾として  
 
アコン  ルウェ  ネ ア